In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('stocks_data.xlsx')

In [3]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,34.664848,35.713787,34.265255,35.214291,14.719626,5466961
1,2010-01-05,35.813683,37.112370,34.964546,36.413078,15.220719,8647066
2,2010-01-06,36.712772,36.862621,35.563938,36.113380,15.095444,3697405
3,2010-01-07,36.213280,36.812672,35.663837,36.363129,15.199841,3938096
4,2010-01-08,36.562923,37.511963,36.163330,36.363129,15.199841,4779058
...,...,...,...,...,...,...,...
3079,2022-06-24,30.500000,31.500000,28.500000,29.600000,29.600000,20057294
3080,2022-06-27,29.650000,31.049999,28.500000,31.000000,31.000000,13650444
3081,2022-06-28,31.850000,32.549999,31.250000,32.549999,32.549999,3840227
3082,2022-06-29,33.900002,34.150002,33.150002,34.150002,34.150002,2092285


In [4]:
df['Adj Open'] = df['Open'] - df['Open']*(df['Close']-df['Adj Close'])/df['Close']

In [5]:
df.drop(columns=['Open', 'High', 'Low', 'Close'], inplace=True)

In [6]:
df.rename(columns = {'Adj Close':'close', 'Adj Open':'open'}, inplace = True)

In [7]:
df['return'] = np.log(df['close']/df['open'])

In [8]:
# Moving Average volume
df['mvolume'] = df.Volume.rolling(3).mean()
df['mvolume'] = df['mvolume'].shift(1)

In [9]:
# Moving Average price
df['mprice'] = df.close.rolling(3).mean()
df['mprice'] = df['mprice'].shift(1)

In [10]:
df.head(20)

,Date,close,Volume,open,return,mvolume,mprice
0,2010-01-04,14.719626,5466961,14.489959,1.572583e-02,NaN,NaN
1,2010-01-05,15.220719,8647066,14.970171,1.659799e-02,NaN,NaN
2,2010-01-06,15.095444,3697405,15.345990,-1.646127e-02,NaN,NaN
3,2010-01-07,15.199841,3938096,15.137204,4.129418e-03,5.937144e+06,15.011930
4,2010-01-08,15.199841,4779058,15.283356,-5.479394e-03,5.427522e+06,15.172002
5,2010-01-11,15.199841,3872553,15.429509,-1.499686e-02,4.138186e+06,15.165042
6,2010-01-12,14.928413,3064967,15.325112,-2.622643e-02,4.196569e+06,15.199841
7,2010-01-13,14.803139,2221438,14.656987,9.922073e-03,3.905526e+06,15.109365
8,2010-01-14,15.053688,3996886,14.991051,4.169593e-03,3.052986e+06,14.977131
9,2010-01-15,14.970170,1993485,15.178960,-1.385072e-02,3.094430e+06,14.928413


In [11]:
df.isna().sum()

Date       0
close      0
Volume     0
open       0
return     0
mvolume    3
mprice     3
dtype: int64

In [12]:
df.dropna(inplace=True)

In [13]:
# generate signal if current volume below last 3 days of average volume and current price below last 3 days of average price, short sell

df['signal'] = ((df.Volume < df.mvolume) & (df.close < df.mprice))
df['shortsignal'] = 0  
df.loc[df.signal,'shortsignal']= -1  
df['pnl'] = df['return'] * df.shortsignal.shift(1)
df.pnl.sum()

5.154441510660151

In [14]:
df[df.shortsignal == -1]

,Date,close,Volume,open,return,mvolume,mprice,signal,shortsignal,pnl
6,2010-01-12,14.928413,3064967,15.325112,-0.026226,4.196569e+06,15.199841,True,-1,-0.000000
7,2010-01-13,14.803139,2221438,14.656987,0.009922,3.905526e+06,15.109365,True,-1,-0.009922
10,2010-01-18,14.928413,1748031,14.970170,-0.002793,2.737270e+06,14.942332,True,-1,-0.000000
11,2010-01-19,14.719626,2005302,14.928417,-0.014085,2.579467e+06,14.984090,True,-1,0.014085
12,2010-01-20,14.531715,1616753,14.803141,-0.018506,1.915606e+06,14.872737,True,-1,0.018506
...,...,...,...,...,...,...,...,...,...,...
3069,2022-06-10,43.549999,928567,43.450001,0.002299,1.813098e+06,44.216666,True,-1,-0.002299
3070,2022-06-13,41.849998,1054864,42.900002,-0.024780,1.406379e+06,43.749999,True,-1,0.024780
3073,2022-06-16,37.750000,2052666,40.450001,-0.069081,2.331386e+06,40.483334,True,-1,-0.000000
3075,2022-06-20,34.150002,1271932,35.099998,-0.027438,2.601387e+06,37.783334,True,-1,-0.000000


In [15]:
#### cumulative PNL
df.pnl.iloc[[0]] = 1
df['cumpnl'] = df.pnl.cumsum()

C:\Users\Admin\AppData\Local\Temp\ipykernel_11980\2953402727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.pnl.iloc[[0]] = 1


In [16]:
df

,Date,close,Volume,open,return,mvolume,mprice,signal,shortsignal,pnl,cumpnl
3,2010-01-07,15.199841,3938096,15.137204,0.004129,5.937144e+06,15.011930,False,0,1.000000,1.000000
4,2010-01-08,15.199841,4779058,15.283356,-0.005479,5.427522e+06,15.172002,False,0,-0.000000,1.000000
5,2010-01-11,15.199841,3872553,15.429509,-0.014997,4.138186e+06,15.165042,False,0,-0.000000,1.000000
6,2010-01-12,14.928413,3064967,15.325112,-0.026226,4.196569e+06,15.199841,True,-1,-0.000000,1.000000
7,2010-01-13,14.803139,2221438,14.656987,0.009922,3.905526e+06,15.109365,True,-1,-0.009922,0.990078
...,...,...,...,...,...,...,...,...,...,...,...
3079,2022-06-24,29.600000,20057294,30.500000,-0.029952,4.022404e+06,31.583333,False,0,-0.000000,6.154442
3080,2022-06-27,31.000000,13650444,29.650000,0.044525,9.048207e+06,30.383333,False,0,0.000000,6.154442
3081,2022-06-28,32.549999,3840227,31.850000,0.021740,1.272952e+07,30.200000,False,0,0.000000,6.154442
3082,2022-06-29,34.150002,2092285,33.900002,0.007348,1.251599e+07,31.050000,False,0,0.000000,6.154442


In [17]:
df['cumpnl'] = df.cumpnl - 1

In [30]:
df[df.pnl != 0]

,Date,close,Volume,open,return,mvolume,mprice,signal,shortsignal,pnl,cumpnl
3,2010-01-07,15.199841,3938096,15.137204,0.004129,5.937144e+06,15.011930,False,0,1.000000,0.000000
7,2010-01-13,14.803139,2221438,14.656987,0.009922,3.905526e+06,15.109365,True,-1,-0.009922,-0.009922
8,2010-01-14,15.053688,3996886,14.991051,0.004170,3.052986e+06,14.977131,False,0,-0.004170,-0.014092
11,2010-01-19,14.719626,2005302,14.928417,-0.014085,2.579467e+06,14.984090,True,-1,0.014085,-0.000007
12,2010-01-20,14.531715,1616753,14.803141,-0.018506,1.915606e+06,14.872737,True,-1,0.018506,0.018499
...,...,...,...,...,...,...,...,...,...,...,...
3070,2022-06-13,41.849998,1054864,42.900002,-0.024780,1.406379e+06,43.749999,True,-1,0.024780,5.058196
3071,2022-06-14,39.900002,2927253,41.500000,-0.039317,1.059739e+06,43.083332,False,0,0.039317,5.097513
3074,2022-06-17,35.900002,2739455,37.000000,-0.030181,2.663987e+06,39.116667,False,0,0.030181,5.127694
3076,2022-06-21,33.200001,4979885,33.000000,0.006042,2.021351e+06,35.933334,False,0,-0.006042,5.121652


In [19]:
len(np.where(df['shortsignal'] == -1)[0])

1075

In [20]:
len(np.where(df['pnl']<0)[0])

392

In [21]:
pnl_dict = {}

In [22]:
pnl_dict['company_name'] = df['pnl'].sum()

In [23]:
new_df = df.drop(columns=['close', 'Volume', 'open', 'return', 'mvolume', 'mprice', 'signal', 'shortsignal', 'cumpnl'])

In [24]:
new_df['Date'] = pd.to_datetime(new_df['Date'], errors='coerce')
new_df = new_df.groupby(pd.Grouper(key='Date', axis=0, freq='y')).sum()

In [25]:
new_df.rename(columns = {'pnl':'company_name'}, inplace = True)

In [26]:
new_df

,company_name
Date,
2010-12-31,1.668898
2011-12-31,0.910485
2012-12-31,0.263448
2013-12-31,0.252353
2014-12-31,0.110152
2015-12-31,-0.048276
2016-12-31,0.949430
2017-12-31,0.207780
2018-12-31,-0.161038


In [27]:
new_df.company_name.sum()

6.154441510660151